## GMW 5204 Vector Projected Field Electromagnet Training
1. This notebook includes the sample code of the data landscape plotting, training process and examination of field scan and angle scan.
2. The module used for training is Keras, Tensorflow.
3. To examine the model with field or angle scan, please remember to standardize the data before predicting the results.
4. The sensitivity of the 3T Senis sensor used for data collection is 5V/T. To predict the applied voltages of Kepcos, the input field should be divided by 2000 (Oe/V).
5. The computer system used for training is 2015 MacBook Pro with 2.7 GHz dual-core Intel Core i5 CPU.

In [ ]:
# Plotting the contour of training set
%matplotlib inline 

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [8, 6]

theta = np.linspace( 0 , 2 * np.pi , 150 )
radius = 2000
a = radius * np.cos( theta )
b = radius * np.sin( theta )
#plt.plot(a, b, c='k')

data = np.loadtxt("0927_train.csv",skiprows=1,delimiter=',')
plt.scatter(data[:,3]*2000,data[:,4]*2000, c= data[:,5]*2000,s=4,cmap='bwr')
cbar = plt.colorbar()
cbar.ax.set_title('$H_z$ (Oe)',fontsize = 15)
cbar.ax.tick_params(labelsize=12)
plt.xlabel('$H_x$ (Oe)',fontsize = 16)
plt.ylabel('$H_y$ (Oe)',fontsize = 16)
plt.xlim(-3500,3500)
plt.ylim(-3200,3200)
plt.xticks(fontsize= 13)
plt.yticks(fontsize= 13)

In [ ]:
# Training the data set
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#Load the training set then standardize and randomly shuffle the data
data = np.loadtxt("0928_train_weighted.csv",skiprows=1,delimiter=',')
scaler = preprocessing.StandardScaler()
np.random.shuffle(data)

Field = data[:,3:6]  # Measured Field
Voltage = data[:,0:3]  # Applied Voltage
H = scaler.fit(Field).transform(Field)
std_H = scaler.fit(Field).scale_
avg_H = scaler.fit(Field).mean_
V = scaler.fit(Voltage).transform(Voltage)
std_V = scaler.fit(Voltage).scale_
avg_V = scaler.fit(Voltage).mean_


train_x, test_x, train_y, test_y = train_test_split(H, V, test_size=0.2, random_state=0)

#Create Keras model
model1 = Sequential()
model1.add(Dense(18, input_dim=3,activation='tanh',kernel_initializer='random_normal'))
model1.add(Dense(3, activation='linear',kernel_initializer='random_normal'))
          
# Gradient descent algorithm
adam= Adam(0.04)

model1.compile(loss='mse', optimizer= adam)
history = model1.fit(train_x, train_y, validation_data=(test_x,test_y), batch_size= 6000,epochs=1000)

print('std_H:',std_H)
print('avg_H:',avg_H)
print('std_V:',std_V)
print('avg_V:',avg_V)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("No. of Iterations")
plt.ylabel("J(Theta1 Theta0)/Cost")
plt.show()


In [ ]:
# Examination of Angle Scan
field = 2000
anglist = np.arange(0,361,3)
Vlist = np.zeros((121,3))
for i in range(121):
    ang = np.pi/180*i*3
    Vlist[i,:]= model1.predict((np.array([[field/2000*np.cos(ang),field/20000*np.sin(ang),0]])-avg_H)/std_H)*std_V+avg_V

plt.scatter(anglist, Vlist[:,0])
plt.scatter(anglist, Vlist[:,1])
plt.scatter(anglist, Vlist[:,2])
#plt.ylim(-10.5,10.5)
plt.xlabel('Applied Field Angle (Deg)',fontsize = 16)
plt.ylabel('Applied Voltage (V)',fontsize = 16)

plt.show()

In [ ]:
# Examination of Field Scan
field = 2000
step = 20
Hlist = np.arange(-field,field+step,step)
Vlist = np.zeros((2*int(field/step)+1,3))
for i in np.arange(np.shape(Hlist)[0]):
    Vlist[i,:]= model1.predict((np.array([[0,Hlist[i]/2000,0]])-avg_H)/std_H)*std_V+avg_V
                                         
plt.scatter(Hlist, Vlist[:,0])
plt.scatter(Hlist, Vlist[:,1])
plt.scatter(Hlist, Vlist[:,2])
plt.xlabel('Applied Field (Oe)',fontsize = 16)
plt.ylabel('Applied Voltage (V)',fontsize = 16)
#plt.ylim(-10.5,10.5)

plt.show()